In [4]:
import os

import pandas as pd
import geopandas as gpd
from shapely.geometry import box

PATH_TO_INVENTORY = "/Users/jamesbyers/code/github/Kaggle/openai_to_z/data/metadata/cms_brazil_lidar_tile_inventory.csv"

df = pd.read_csv(PATH_TO_INVENTORY)

In [7]:
df.columns

Index(['filename', 'max_lat', 'min_lat', 'max_lon', 'min_lon', 'file_type',
       'file_size_mb', 'file_format', 'version', 'created', 'utmzone', 'srs',
       'geometry'],
      dtype='object')

In [13]:
# Create the geometry column (bounding box polygons)
df['geometry'] = df.apply(
    lambda row: box(row['min_lon'], row['min_lat'], row['max_lon'], row['max_lat']),
    axis=1
)

# Make a GeoDataFrame, set CRS to WGS84
gdf = gpd.GeoDataFrame(df, geometry='geometry', crs='EPSG:4326')

# Write to GPKG
gdf.to_file('tiles.gpkg', layer='tiles', driver='GPKG')

print("GeoPackage file 'tiles.gpkg' created.")

GeoPackage file 'tiles.gpkg' created.


In [11]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import box

records = []
for i, row in df.iterrows():
    proj4 = row['srs']
    if pd.isnull(proj4) or not str(proj4).strip():
        print(f"Row {i}: SRS missing!")
        continue
    try:
        temp_gdf = gpd.GeoDataFrame(
            [row],
            geometry=[box(row['min_lon'], row['min_lat'], row['max_lon'], row['max_lat'])],
            crs=proj4   # Use the full proj string
        )
        temp_gdf = temp_gdf.to_crs("EPSG:4326")  # convert to common CRS, e.g. WGS84
        records.append(temp_gdf)
    except Exception as e:
        print(f"Row {i}: Could not interpret SRS ({proj4}) - {e}")

In [12]:
from pyproj import CRS

crs_obj = CRS(row['srs'])
print(crs_obj.to_epsg())  # Will return the EPSG code if possible, or None

None
